In [2]:
!pip install python-igraph leidenalg

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------  1.6/1.6 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 10.5 MB/s eta 0:00:00


In [1]:
import sys
import os
import networkx as nx
from collections import Counter
import igraph as ig
from leidenalg import find_partition, ModularityVertexPartition

import sys
project_path = os.getcwd().split("Code")[0]
sys.path.append(project_path)

# Importa la classe GraphConstructor
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Funzione per convertire un grafo NetworkX in un grafo iGraph con mappatura dei nodi
def convert_to_igraph(graph):
    """
    Converte un grafo NetworkX in un grafo iGraph, mappando i nodi a indici consecutivi.
    
    Args:
        graph (networkx.Graph): Il grafo di input.
    
    Returns:
        igraph.Graph: Il grafo convertito in formato iGraph.
        dict: La mappatura dei nodi di NetworkX agli indici di iGraph.
    """
    # Crea una mappatura dei nodi a indici consecutivi
    node_mapping = {node: idx for idx, node in enumerate(graph.nodes())}
    
    # Converti in un grafo iGraph
    g = ig.Graph(directed=graph.is_directed())
    g.add_vertices(len(graph.nodes))  # Aggiungi nodi in base alla mappatura
    g.add_edges([(node_mapping[u], node_mapping[v]) for u, v in graph.edges])  # Usa gli indici mappati
    
    return g, node_mapping

# Funzione per eseguire il clustering con l'algoritmo di Leiden
def leiden_clustering(graph):
    """
    Esegue il clustering sul grafo utilizzando l'algoritmo di Leiden.
    
    Args:
        graph (networkx.Graph): Il grafo su cui eseguire il clustering.
    
    Returns:
        dict: Un dizionario con l'etichetta del cluster per ciascun nodo.
    """
    # Converte il grafo in formato iGraph e ottieni la mappatura dei nodi
    ig_graph, node_mapping = convert_to_igraph(graph)
    
    # Esegui il Leiden Algorithm con modularità come funzione obiettivo
    partition = find_partition(ig_graph, ModularityVertexPartition)
    
    # Inverti la mappatura dei nodi per creare il dizionario dei cluster
    inverse_mapping = {idx: node for node, idx in node_mapping.items()}
    cluster_labels = {inverse_mapping[node]: cluster_id 
                      for cluster_id, community in enumerate(partition) 
                      for node in community}
    
    return cluster_labels

# Funzione per sanitizzare gli attributi del grafo (nodi e archi)
def sanitize_graph_attributes(graph):
    """
    Sanitizza le chiavi e i valori degli attributi dei nodi e degli archi
    per renderli compatibili con il formato GML.
    """
    for node in graph.nodes:
        attrs = graph.nodes[node]
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.nodes[node].clear()
        graph.nodes[node].update(sanitized_attrs)

    for u, v, attrs in graph.edges(data=True):
        sanitized_attrs = {}
        for key, value in attrs.items():
            # Rimpiazza spazi e trattini nei nomi delle chiavi
            new_key = str(key).replace(" ", "_").replace("-", "_")
            # Converte tutti i valori in stringhe
            sanitized_attrs[new_key] = str(value) if not isinstance(value, str) else value
        graph.edges[u, v].clear()
        graph.edges[u, v].update(sanitized_attrs)

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph

# Esegui il clustering con il Leiden Algorithm
leiden_labels = leiden_clustering(graph)

# Copia il grafo originale per assegnare i cluster
gr = graph.copy()

# Assegna il cluster a ciascun nodo
for node, label in leiden_labels.items():
    gr.nodes[node]['cluster'] = label  # Assegniamo l'etichetta del cluster al nodo

# Sanitizza gli attributi del grafo prima di esportarlo
sanitize_graph_attributes(gr)

# Esporta il grafo in formato GML
output_path = "graph_with_leiden_clusters.gml"
nx.write_gml(gr, output_path)
